In [33]:
import pandas as pd
import numpy as np
from sklearn import *
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB1
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [35]:
class main_class():
    def __init__(self,root):
        self.file_path=root
    def img_prep(self):
        img_path=[]
        img_name=[]
        img_lab=[]
        for root,dirts,file in os.walk(self.file_path):
            for files in file:
                path_img=os.path.join(root,files)
                name_img=path_img.split('/')[5]
                labl_img=(1 if name_img=='MildDemented' 
                          else 2 if name_img=='ModerateDemented' 
                          else 3 if name_img=='NonDemented'
                          else 0)
                img_path.append(path_img)
                img_name.append(name_img)
                img_lab.append(labl_img)
               
        df=pd.DataFrame(list(zip(img_path, img_name,img_lab)),columns=['Link','Name','Lable'])
        return df 

In [36]:
img_class=main_class('../input/alzheimers-disease-dataset')
df=img_class.img_prep()

In [37]:
x=df.Link
y=df.Lable
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.01,random_state=101)

In [38]:
X_train=[]
for i in xtrain:
    img_pre=cv2.imread(i)
    img_pre=cv2.resize(img_pre, (150, 150))
    X_train.append(img_pre)
    
X_train=np.array(X_train)
ytrain=to_categorical(ytrain)
ytest=to_categorical(ytest)

In [39]:
efficient = EfficientNetB0(weights='imagenet',include_top = False, input_shape=(150, 150, 3)) 

In [40]:
model = efficient.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(0.5)(model)

model = tf.keras.layers.Dense(4, activation = "softmax")(model)

model = tf.keras.models.Model(inputs = efficient.input , outputs = model)

In [41]:
model.compile(loss = 'categorical_crossentropy', 
             optimizer = "Adam", 
              metrics=["accuracy"]
             )

In [42]:
tensorboard = TensorBoard(log_dir="logs")
checkpoint = ModelCheckpoint("efficient.h5", 
                            monitor='val_accuracy', 
                            mode = "auto", 
                            verbose = 1,
                            save_best_only =True)
reduce_LR = ReduceLROnPlateau(monitor="val_accuracy", 
                             factor = 0.3, 
                             patience = 2, 
                             min_delta=0.001, 
                             mode = "auto", 
                            verbose = 1)

In [43]:
history = model.fit(X_train, ytrain,
                  validation_split = 0.1, 
                  epochs = 10, 
                  verbose =1, 
                  batch_size = 32, 
                  callbacks = [tensorboard, checkpoint, reduce_LR])

# <h2>val_accuracy improved from 0.98423 to **0.98580**</h2>